In [8]:
import numpy as np    #import numpy
from math import factorial as f    #import math

In [9]:
def nCk(n,k):    #define a function to compute the combinations
    """A function to compute n choose k.  The function returns 0 if k>n."""
    try:
        r = f(n)/(f(k)*f(n-k))
    except:
        r = 0.0
    return r


In [10]:
def B(k,n,p):    #define a function to compute the binomial probability mass function
    """A function to compute the probability of k successes given n trials w/ prob p of success."""
    return nCk(n,k)*p**k*(1 - p)**(n - k)


In [11]:
def pmf(k,n_1,p_1,n_2,p_2):    #define a function to compute the derived pmf
    """A function to compute the probability distribution over the number of hits remaining after defending.
    The function takes in:
    k: nmber of hits
    n_1: the number of attacks
    p_1: the likelihood of an attack being successful
    n_2: the number of defenses
    p_2: the likelhood of defending
    The function returns p(k)."""
    p = 0.0
    if k >= 0:
        for i in range(0,n_1):
            p += B(k+i,n_1,p_1)*B(i,n_2,p_2)
    if k < 0:
        for i in range(0,n_2):
            p += B(i,n_1,p_1)*B(k+i,n_2,p_2)
    return p


In [12]:
def cmf(Z,n_1,p_1,n_2,p_2):    #a function to compute the cmf - in this case prob that 1 kills 2
    """A function to compute the probability that computes the CDF of the PMF in the previous cell.
    Substantively, this would be the probability that 1 defeats 2.
    The function takes in:
    Z: the number of hits needed to fell the target
    n_1: the number of attack against the target
    p_1: the likelihood that an attack will be successful
    n_2: the number of defenses
    p_2: the likelihood of defending
    The function returns p(k>=Z)."""
    c = 0.0
    for k in range(Z,n_1+1):
        c += pmf(k,n_1,p_1,n_2,p_2)
    return c


In [13]:
def Cost(shoot,assault,armor,stamina,bulk,n_action,armor_target,stamina_target):
    """A function to compute the cost of a unit given assumptions about the importance of ranged fire vs close combat.
    The function takes in:
    shoot: the number of shooting attacks of the unit to be evaluated
    assault: the number of close combat attacks of the unit to be evaluated
    armor: the armor of the unit to be evaluated
    stamina: the stamina of the unit to be evaluated
    bulk: the number of individuals who can fit in a discrete space
    n_action: the number of actions the unit to be evaluated can make per turn
    armor_target: the armor of the basic target - assume the same for all units to be evaluated
    stamina_target: the stamina of the basic target - assume the same for all units to be evaluated
    The function returns a non-normalized number that represents a weighted combination of the probability that the
    unit to be evaluated kills the basic target and the probability that a unit of the basic target kills the
    unit to be evaluated."""
    #fixed model parameters
    p_hit = 2.0/6.0    #from the dice, the prob of a hit
    p_chit = 1.0/6.0    #from the dice, the prob of a critical hit
    p_defend = 2.0/6.0    #from the dice, the prob of a block

    w_shoot = 0.5    #the relative importance of shooting - must add to one with assault
    w_assault = 0.5    #the relative importance of assault - must add to one with shooting
    w_offense = 0.5    #the relative importance of offense - must add to one with defence
    w_defense = 0.5    #the relative importance of defense - must add to one with offense

    Shoot_base = 6    #the number of base shooting attacks
    Assault_base = 6    #the number of base assault attacks

    hex_size = 3.0    #how much can fit in a hex
    
    c = n_action*(w_offense*(w_shoot*hex_size/bulk*cmf(stamina_target,shoot,p_hit+p_chit,armor_target,p_defend) 
                             + w_assault*hex_size/bulk*cmf(stamina_target,assault,p_hit+p_chit,armor_target,p_defend)) 
                  + w_defense*(w_shoot*(1 - cmf(stamina,Shoot_base,p_hit+p_chit,armor,p_defend)) 
                               + w_assault*(1 - cmf(stamina,Assault_base,p_hit+p_chit,armor,p_defend))))     #compute the cost
    
    return c


In [21]:
#Individual baseline cost with no weapons
Cost_Astartes_base = Cost(0,2,2,2,1,2,2,2)
M = 10/Cost_Astartes_base
print('The normalizing constant is:',M,'.  With this constant an individual basic Astartes is 10 pts.')


The normalizing constant is: 22.677165354330718 .  With this constant an individual basic Astartes is 10 pts.


In [24]:
#special rule costs
Jump_infantry = Cost(0,2,2,2,1,3,2,2) - Cost_Astartes_base    #assume more actions
#Hardened_Armor rerolls
Counterattack = Cost(0,2,2,2,1,2.3,2,2) - Cost_Astartes_base    #assume more actions
#Scout = #assume more actions
#Stealth rerolls
#Shrouded rerolls
Slow_and_Purposeful = Cost(0,2,2,2,1,3,2,2) - Cost_Astartes_base    #assume more actions
Chain_fire = 
#Lone_Killer no idea
#Infiltrate = #assume more actions
#Outflank = #assume more actions 

print(round(M*Jump_infantry))
print(round(M*Counterattack))

5
1


In [ ]:
#Costs for individual soldiers
#Headquarters
Centurion = Cost(0,2,4,3,1,2,2,2)    #Fearless

#Elites
Veteran = Cost(0,2,3,2,1,2,2,2)    #one of Fearless, Sniper, Furious Charge, Outflank, or Tank Hunters Special rule
Destroyer = Cost(0,2,2,2,1,2,2,2)    #counterattack special rule
#Cataphractii = #Hardened Armor
#Tartaros = 

#Troops
Tactical = Cost(0,2,2,2,1,2,2,2)    #Fury of the Legion
Recon = Cost(0,2,2,2,1,2,2,2)    #Infiltrate, Scout, Outflank, and Acute Senses
Breacher = Cost(0,2,3,2,1,2,2,2)    #Hardened Armor
Assault = Cost(0,2,2,2,1,2,2,2)    #Jump Infantry

#Support
Support = Cost(0,2,2,2,1,2,2,2)
Seeker = Cost(0,2,2,2,1,2,2,2)    #Preferred Enemy

In [ ]:
#Shooting Weapon cost factoring in critical effect
Cost_bolter = Cost(2,2,2,2,1,2,2,2) + (1 - 5.0/6.0**2.0)*0.5*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base
Cost_bolt_pistol = Cost_bolter*1.0/2.0
Cost_combibolter = Cost(4,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**4.0)*0.5*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base
Cost_heavy_bolter = Cost(6,2,2,2,1,2,2,2) + (1 - 5.0/6.0**6.0)*0.5*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base
Cost_meltagun = 5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*Cost(3,2,2,2,1,2,0,2) - Cost_Astartes_base
Cost_volkite_caliver = 5.0/6.0**4.0*Cost(4,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**4.0)*Cost(4,2,2,2,1,2,2,0) - Cost_Astartes_base
Cost_volkite_serpenta = 1.0/2.0*(5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*Cost(3,2,2,2,1,2,2,0) - Cost_Astartes_base)
Cost_flamer = 1.0/2.0*(5.0/6.0**4.0*Cost(4,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**4.0)*2.0*Cost(4,2,2,2,1,2,2,2) - Cost_Astartes_base)
Cost_hand_flamer = 1.0/2.0*(5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*2.0*Cost(3,2,2,2,1,2,2,2) - Cost_Astartes_base)
Cost_heavy_flamer = 1.0/2.0*(5.0/6.0**6.0*Cost(6,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**6.0)*2.0*Cost(6,2,2,2,1,2,2,2) - Cost_Astartes_base)
Cost_plasmapistol = 1.0/2.0*(5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*((B(0.0,4.0,1.0/6.0) + B(1.0,4.0,1.0/6.0))*Cost(7,2,2,2,1,2,2,2) - (B(2.0,4.0,1.0/6.0) + B(3.0,4.0,1.0/6.0) + B(4.0,4.0,1.0/6.0))*Cost(3,2,2,2,1,2,2,2)) - Cost_Astartes_base)
Cost_plasmagun = (5.0/6.0**3.0*Cost(3,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**3.0)*((B(0.0,4.0,1.0/6.0) + B(1.0,4.0,1.0/6.0))*Cost(7,2,2,2,1,2,2,2) - (B(2.0,4.0,1.0/6.0) + B(3.0,4.0,1.0/6.0) + B(4.0,4.0,1.0/6.0))*Cost(3,2,2,2,1,2,2,2)) - Cost_Astartes_base)
Cost_combiweapon = 1.0/2.0*Cost_bolter + 1.0/2.0*(1.0/3.0*Cost_flamer + 1.0/3.0*Cost_plasmagun + 1.0/3.0*Cost_meltagun)
Cost_sniper_rifle = Cost(4,2,2,2,1,2,2,2) + (1 - 5.0/6.0**4.0)*1.0/3.0*1.0/2.0*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base
Cost_rocket_launcher = 5.0/6.0**5.0*Cost(5,2,2,2,1,2,2,2) + (1.0 - 5.0/6.0**5.0)*Cost(7,2,2,2,1,2,2,0) - Cost_Astartes_base
Cost_graviton_gun = Cost(6,2,2,2,1,2,2,2) + (1 - 5.0/6.0**6.0)*Cost(2,2,2,2,1,2,2,2) - Cost_Astartes_base

In [ ]:
#Item costs
Cataphractii_terminator_armor = Cost(0,3,5,3,1.5,2,2,2) - Cost_Astartes_base
Tartaros_terminator_armor = Cost(0,3,5,3,1.5,2,2,2) - Cost_Astartes_base
#Artificer_armor = 
#Iron_halo = 
#Breacher_shield = 
